In [1]:
import json
import os
import numpy as np
import random
import torch
from torch.optim import Adam
import dgl

from src import *
from pprint import pprint


# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(f'device: {device}')

In [2]:
from argparse import ArgumentParser
parser = ArgumentParser()

import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description="Parser for model configuration")

parser.add_argument('--feature_file', type=str, default="./data/feature_dict.json", help='feature json file')
parser.add_argument('--reuse_rate_file', type=str, default="./data/reuse_rate_dict.json", help='reuse rate json file')
# parser.add_argument('--setting', type=str, required=True, help='graph learning setting: inductive/transductive')
# parser.add_argument('--model_path', type=str, required=True, default="./model/", help='model file path')
parser.add_argument('--output_dir', type=str, default="./output/", help='output path')
parser.add_argument('--agg_type', type=str, default="attn", help='aggregation type')
parser.add_argument('--nclient', type=int, default=4, help='number of clients')
parser.add_argument('--valid', type=float, default=0.2, help='split ratio of validation set')
parser.add_argument('--test', type=float, default=0.2, help='split ratio of test set')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout ratio')
parser.add_argument('--random_seed', type=int, default=1, help='random seed for initialization')
parser.add_argument('--relu', type=float, default=0.2, help='ReLU threshold')
parser.add_argument('--reuse_th', type=float, default=0.5, help='threshold for reuse')
parser.add_argument('--batch_size', type=int, default=8192, help='batch size for training')
parser.add_argument('--embed_size', type=int, default=256, help='size of the embedding layer')
parser.add_argument('--hidden_size', type=int, default=256, help='size of the hidden layer')
parser.add_argument('--gnn_depth', type=int, default=2, help='depth of the GNN')
parser.add_argument('--max_lr', type=float, default=0.001, help='maximum learning rate')
parser.add_argument('--warmup', type=float, default=0.1, help='warmup ratio for learning rate')
parser.add_argument('--max_epoch', type=int, default=200, help='maximum number of epochs')
parser.add_argument('--early_stop', type=int, default=40, help='early stopping epochs')
parser.add_argument('--device', type=int, default=0, help='GPU ID')

args = parser.parse_args(args=[])
pprint(args)

args.device = torch.device(f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
dgl.seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)

Namespace(feature_file='./data/feature_dict.json', reuse_rate_file='./data/reuse_rate_dict.json', output_dir='./output/', agg_type='attn', nclient=4, valid=0.2, test=0.2, dropout=0.5, random_seed=1, relu=0.2, reuse_th=0.5, batch_size=8192, embed_size=256, hidden_size=256, gnn_depth=2, max_lr=0.001, warmup=0.1, max_epoch=200, early_stop=40, device=0)


In [3]:
with open(args.feature_file, 'r') as f:
    feature_dict = json.load(f)

with open(args.reuse_rate_file, 'r') as f:
    reuse_rate_dict = json.load(f)
    
P = PassREfinder(feature_dict, reuse_rate_dict, args)

P.print_graph()

train_loader_list = P.get_train_data_loader_list()
train_nfeat_list = P.pop_train_node_feature()

valid_loader = P.get_data_loader('valid')
valid_nfeat = P.pop_node_feature('valid')

client_list = []
for i in range(args.nclient):
    client_list.append(Client(train_loader_list[i], train_nfeat_list[i], args))
train(client_list, valid_loader, valid_nfeat, args)
# evaluate(model, test_loader, test_nfeat, device)

Constructing graph...
Constructing graph... Done
Splitting graph...
Splitting graph... Done
---------------------Data statistics---------------------
#[Train] nodes: [1489, 1489, 1489, 1489], edges: [28076, 24164, 29070, 23326], target edges: [28076, 24164, 29070, 23326]
#[Valid] nodes: 1985, edges: 53672, target edges: 53672
#[Test]  nodes: 1986, edges: 49372, target edges: 49372
---------------------------------------------------------

2024-08-21 01:32:00 start
2024-08-21 01:32:07 Epoch [1/200], Train Loss: 0.7013, Valid Loss: 0.6927, Precision: 0.5734, Recall: 0.0076, F1: 0.0150
2024-08-21 01:32:10 Epoch [2/200], Train Loss: 0.6934, Valid Loss: 0.6934, Precision: 0.5060, Recall: 0.9952, F1: 0.6709
2024-08-21 01:32:14 Epoch [3/200], Train Loss: 0.6842, Valid Loss: 0.6980, Precision: 0.5070, Recall: 1.0000, F1: 0.6728
2024-08-21 01:32:18 Epoch [4/200], Train Loss: 0.6774, Valid Loss: 0.7114, Precision: 0.5070, Recall: 1.0000, F1: 0.6728
2024-08-21 01:32:21 Epoch [5/200], Train Loss: 